In [38]:
import pandas as pd
import numpy as np
import sklearn
import os
import datetime
import time
import git
import sys
import math
from geopy.distance import geodesic
from sklearn.cluster import _kmeans

In [25]:
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir
datadir = f"{homedir}" + "/models/processing/USA/County_Based/"

In [41]:
#helper functions
def logFunc(x):
    if x < 0.01:
        x = 0.01
    return math.log10(x)

# Convert longitude latitude pair to x, y, z Cartesian coordinates\n
def convertPts(pair):
    lon = pair[0]
    lat = pair[1]
    R = 3958.8
    lonRad = lon * math.pi / 180
    latRad = lat * math.pi / 180
    x = R * math.cos(latRad) * math.cos(lonRad)
    y = R * math.cos(latRad) * math.cos(lonRad)
    z = R * math.sin(lat)
    return (x, y, z)

def getX(x):
    return x[0]

def getY(x):
    return x[1]

def getZ(x):
    return x[2]

In [27]:
#Neighbor Data
neighborcounties = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/neighborcounties.csv", index_col = 0)

# read in the files, load as dataframe
Age_Race = pd.read_csv(datadir + 'Age_Race_Filled.csv')
Population = pd.read_csv(datadir + 'Total_Pop')
Density = pd.read_csv(datadir + 'Density.csv')
JHU = pd.read_csv(datadir + 'aggregate_jhu_filled.csv')
Berkeley = pd.read_csv(datadir + 'Aggregate_Berkeley.csv')
Policies = pd.read_csv(datadir + 'Policy_Transit.csv')
Geography = pd.read_csv(datadir + 'County_Centers.csv')

Data = pd.DataFrame()

DataBasic = pd.DataFrame()
DataDemographics = pd.DataFrame()
DataHealth = pd.DataFrame()
DataGeography = pd.DataFrame()

In [28]:
Data['FIPS'] = Age_Race['fips']

# fix population
Population.columns = ['fips', 'Population']

# drop US territories, train separate models for them
FipsSet = []
counter = 0
for row in Data.iterrows():
    row = row[1][0]
    if math.floor(row / 1000) > 56:
        Data = Data.drop([counter], axis = 0)
    else:
        FipsSet.append(float(row))
    counter += 1
    
# edit county centers
counter1 = 0
for row in Geography.iterrows():
    if row[1][1] not in FipsSet:
        Geography = Geography.drop([counter1], axis = 0)
    counter1 += 1

In [29]:
# Nature of the county, includes policies

Data['Pop'] = Population['Population']
Data['Pop'] = Data['Pop'].div(2000000.0) # population max is 5

Data['Density'] = Density['2010 Density per square mile of land area - Population']
Data['Density'] = Data['Density'].div(5000.0) # density max is around 14

Data['Area'] = JHU['Area in square miles - Land area']
Data['Area'] = Data['Area'].div(5000.0) #max around 4

Data['UrbanRural'] = JHU['Rural-urban_Continuum Code_2013']
Data['UrbanRural'] = Data['UrbanRural'].div(1.5) # urban rural max is 7

Data['EconType'] = JHU['Economic_typology_2015']
Data['EconType'] = Data['EconType'].div(1.5) # economic typology max is 3

# Policies
Data['Policies'] = Policies['Score']
Data['Policies'] = Data['Policies'].div(2) # policies max is just above 4.5

# Typical immigration in/out. Proxy for being a sink/source in flows
Data['Movement'] = JHU['R_NET_MIG_2018']
Data['Movement'] = Data['Movement'].div(20.0) #range around -3.5 to 3.5

DataBasic = Data[['Pop', 'Density', 'Area', 'UrbanRural', 'EconType', 'Policies', 'Movement']]

In [30]:
# Demographics of the county

# Age distribution
Data['65+'] = Age_Race['65 to 74 years'] + Age_Race['75 to 84 years'] + Age_Race['85 years and over']
Data['65+'] = Data['65+'] / Population['Population']
Data['65+'] = Data['65+'].mul(2400) # 65+ max is 4.5

# Race/gender
Data['Male'] = Age_Race['Male'] / Population['Population']
Data['Male'] = Data['Male'].mul(450) # male max is 2, generally 1

Data['AfricanAmer'] = Age_Race['Exclusively Black or African American'] + Age_Race['Hispanic or Latino (of any race)!!Puerto Rican']
Data['AfricanAmer'] = Data['AfricanAmer'] / Population['Population']
Data['AfricanAmer'] = Data['AfricanAmer'].mul(9000) # African American max is 6
     
# Politics/education/income/economy
Data['CollegePlus'] = JHU['Percent of adults completing some college or associate\'s degree 2014-18']
Data['CollegePlus'] = Data['CollegePlus'].mul(20) # education max is 3, generally around 1-1.5

Data['Income'] = JHU['Median_Household_Income_2018']
Data['Income'] = Data['Income'].div(40000) # income max is 4, generally around 2  

Data['Unemployed'] = JHU['Unemployment_rate_2018']
Data['Unemployed'] = Data['Unemployed'].div(3) # unemployed max around 3
                         
    # need puerto rico voting patterns
Data['Dems'] = Berkeley['FracDem']
Data['Dems'] = Data['Dems'].mul(5) # Dems max is 5, generally around 1.5

DataDemographics = Data[['Male', 'AfricanAmer', 'CollegePlus', 'Income', 'Unemployed', 'Dems']]

In [31]:
#Health care of the county

Data['HospBeds'] = JHU['Total hospital beds per 1000 people (2019)'] # around 2-3

Data['ICUBeds'] = Berkeley['#ICU_beds']
Data['ICUBeds'] = Data['ICUBeds'] / Population['Population']
Data['ICUBeds'] = Data['ICUBeds'].mul(10) # around 11. Outliers ok, very important statistic

#note: not considering comorbidities
Data['HeartDiseaseMort'] = Berkeley['HeartDiseaseMortality'] 
Data['HeartDiseaseMort'] = Data['HeartDiseaseMort'].div(60) # max is 10, typically around 3-4

Data['StrokeMort'] = Berkeley['StrokeMortality']
Data['StrokeMort'] = Data['StrokeMort'].div(15) # max is 7-8, generally around 5

Data['Diabetes'] = Berkeley['DiabetesPercentage'] 
Data['Diabetes'] = Data['Diabetes'].mul(9) # max 3

Data['Smokers'] = Berkeley['SmokersPercentage'] 
Data['Smokers'] = Data['Smokers'].mul(9) # max 3

DataHealth = Data[['HospBeds', 'ICUBeds', 'HeartDiseaseMort', 'StrokeMort', 'Diabetes', 'Smokers']]

In [32]:
#Geography

Data['pLonLat'] = list(zip(Geography.pclon10, Geography.pclat10)) # population weighted
Data['pLonLat'] = Data['pLonLat'].values

Data['XYZ'] = Data['pLonLat'].apply(convertPts)

Data['xVal'] = Data['XYZ'].apply(getX)
Data['xVal'] = Data['xVal'].div(100)

Data['yVal'] = Data['XYZ'].apply(getY)
Data['yVal'] = Data['yVal'].div(100)

Data['zVal'] = Data['XYZ'].apply(getZ)
Data['zVal'] = Data['zVal'].div(100)

Data = Data.drop(columns=['pLonLat', 'XYZ'])

# approximate distances between counties
# we double count Urban Rural, population, density, size since don't cluster on the entire Data dataframe
# so that nearby urban counties are closer than a rural county adjacent from an urban county
DataGeography = Data[['Pop', 'Density', 'Area', 'UrbanRural', 'xVal', 'yVal', 'zVal']]

In [52]:
# Filling in columns of dataframe by nearest neighbor analysis

for row in Data.iterrows():
    if sum(math.isnan(x) for x in row[1]) == 0:
        neighbors = get_neighbors(row[0])
        for i in range(0, len(row[1]))
        
    #    if math.isnan(entry) == True:
            

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [43]:
for column in Data.columns:
    print((column, Data[column].isnull().sum()))
print(len(Data.columns))

('FIPS', 0)
('Pop', 0)
('Density', 0)
('Area', 21)
('UrbanRural', 21)
('EconType', 21)
('Policies', 0)
('Movement', 21)
('65+', 0)
('Male', 0)
('AfricanAmer', 0)
('CollegePlus', 21)
('Income', 21)
('Unemployed', 21)
('Dems', 0)
('HospBeds', 21)
('ICUBeds', 0)
('HeartDiseaseMort', 0)
('StrokeMort', 1)
('Diabetes', 0)
('Smokers', 0)
('xVal', 0)
('yVal', 0)
('zVal', 0)
24


In [ ]:
# getting specific dataframes for what we want to cluster

In [ ]:
# plot k vs error on a graph, decide optimal k via elbow method

In [ ]:
# plot the clusters across the country to visualize